In [1]:
from brax import envs
from prefacc.training.agents.prefppo import train as prefppo

In [13]:
env = envs.get_environment('inverted_pendulum', backend='spring')

In [14]:
# make_inference_fn, params, _ = prefppo.train(
#   env,
#   num_timesteps=50_000_000, 
#   num_evals=10, 
#   reward_scaling=10, 
#   episode_length=1000, 
#   normalize_observations=True, 
#   action_repeat=1, 
#   unroll_length=5, 
#   num_minibatches=32, 
#   num_updates_per_batch=4, 
#   discounting=0.97, 
#   learning_rate=3e-4, 
#   entropy_cost=1e-2, 
#   num_envs=4096, 
#   batch_size=2048, 
#   seed=1,
#   min_replay_size=2048,
#   num_prefs=1400)

make_inference_fn, params, metrics = prefppo.train(
        env,
        num_timesteps=2_000_000, num_evals=20, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=2048, batch_size=1024, seed=1
    )
print("Training complete")

Training complete


In [15]:
print(metrics)

{'eval/walltime': 8.932333707809448, 'training/sps': np.float64(1170651.6609513492), 'training/walltime': 8.302177429199219, 'training/entropy_loss': Array(-5.9992366e-05, dtype=float32), 'training/policy_loss': Array(-0.00807742, dtype=float32), 'training/total_loss': Array(4.2327924, dtype=float32), 'training/v_loss': Array(4.24093, dtype=float32), 'pearson_correlation': Array([nan], dtype=float32), 'eval/episode_reward': Array(529.3828, dtype=float32), 'eval/episode_reward_std': Array(83.38035, dtype=float32), 'eval/avg_episode_length': Array(529.3828, dtype=float32), 'eval/epoch_eval_time': 0.24978852272033691, 'eval/sps': 512433.47214679164}


In [16]:
from brax.io import model
from brax.io import json
from brax.io import html

In [17]:
model.save_params('/tmp/params', params)

In [18]:
params = model.load_params('/tmp/params')
inference_fn = make_inference_fn(params)

In [19]:
env = envs.create('inverted_pendulum', backend='spring')

In [20]:
import jax

In [21]:
jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.jit(inference_fn)

In [22]:
rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(1000):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  act, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_env_step(state, act)

In [23]:
from IPython.display import HTML, clear_output

In [24]:
HTML(html.render(env.sys.tree_replace({'opt.timestep': env.dt}), rollout))